# Lab 4 - InSAR
As we have seen from the videos and lectures, SAR (Synthetic Aperature RADAR) is a widely-used remote sensing technique for measuring a range of surface properties. This class is about geodetic-type remote sensing measurements, so we will focus on InSAR, or Interferometric SAR. In this lab you will gain some experience accessing, modeling, and interpreting InSAR data. 

## Data sources
There are several places and ways to access InSAR data. Some data is available directly as InSAR: SAR products that have already been pre-processed and interfered to produce an interferogram, two SAR acquisitions that have been differenced. 

### __[1. Alaska Satellite Facility](https://asf.alaska.edu/)__
The __[ASF search page](https://search.asf.alaska.edu/#/)__ interactive webpage allows you to select a region or specify a bounding box, select which products you are interested in, download individual products, or perform a bulk download using a Python script. This site is most useful if you will processing SAR data into InSAR products, but it also contains the ARIA interferograms that we will use in this lesson. This site has SAR/InSAR data available from several different missions, including RADARSAT-1, Sentinel-1, JPL's UAVSAR, ALOS-PALSAR, and some ERS data.
<img src="https://raw.githubusercontent.com/jlmaurer/GE6146/master/notebooks/images/Vertex_homescreen.png" alt="Vertex home page" width="700">

### __[2. ARIA](https://aria.jpl.nasa.gov/)__
The ARIA project is "a collaboration between JPL and Caltech to exploit radar and optical remote sensing, GPS, and seismic observations for hazard science and response." They will eventually provide automatic and real-time processing of SAR, InSAR, and GNSS data to aid in rapid response to all kinds of earth hazards, including earthquakes, volcanoes, large landslides, flooding, storms, etc. Accessing InSAR data using ARIA can be done through ASF or through the __[ARIA-tools](https://github.com/aria-tools/ARIA-tools)__ API, which is what we will use in this lab. 
<img src="https://raw.githubusercontent.com/jlmaurer/GE6146/master/notebooks/images/ARIA_homepage.png" alt="ARIA project home page" width="700">

### __[3. UAVSAR](https://uavsar.jpl.nasa.gov/)__
JPL's airborne SAR sensor is called "UAVSAR" for "Uninhabited Aeriel Vehicle SAR" (although the vehicle almost always actually inhabited!). This mission collects airborne SAR data and makes interferograms available on the __[project website](https://uavsar.jpl.nasa.gov/)__. On their website you can search for and download data for the specific areas where it has been acquired; unlike satellite-based sensors, UAVSAR is on-demand, so it is not acquired everywhere nor all the time. 

## Processing Workflow Considerations
How you process the data to get a result that can be used for geodetic purposes depends on what data you access; i.e.,  Single-Look complex (SLC) products in radar coordinates versus geocoded interferograms. The process you will want to use will depend on several factors, including: 
1. What data is available, and in what format. This includes:  
   - Does the data you want exist?
   - Has it been processed already, and to what level? 
   - Do you need a single pair (interferogram), or do you need to look at time-series?
   - Is the data publically available or will you need to access it through other means (e.g. writing a grant)
   - Does the data exist at the necessary resolution, temporal sampling, etc. for your application?
2. What processing algorithms you have access to use or know how to use?  
   - Some algorithms are open-source and cross-platform, others must be paid for or only work on one OS
   - Some algoirthms are simple and do a few things, others are complex and do many things
   - How precise do your measurements need to be? If your signal is large, you can use simpler processing techniques and still get a useful signal, but if your signal is small you will need to use the best possible methods to perform your analysis. 
3. Do you need time-series analysis or single event pairs? 
4. What computational capabilities (computers, servers, memory, disk space) do you need and do you have access to it?  
   - SAR/InSAR scenes can be large, and processing several can be very computational intensive. 
   - You will need **large memory, processing power, and storage space** capabilities
   - In most cases, access to specialized equipment like many processors or GPUs can significantly aid computational capabilities.  
5. There are many software tools and cutting-edge advanced processing techniques that we will not delve into in this class, but may be useful to you for futher research. Note that some of these are research-grade codes or otherwise not well developed, so learning to use them could take some significant time and effort. Some of these are:  
   - __[The Sentinel-1 toolbox, or SNAP](https://sentinel.esa.int/web/sentinel/toolboxes/sentinel-1)__
   - __[ISCE](https://github.com/isce-framework)__ is a general-purpose processing software for InSAR developed by folks at JPL and elsewhere, written in Python
   - __[GMTSAR](https://topex.ucsd.edu/gmtsar/)__ is a general InSAR processing tool developed by folks at Scripps Institute of Oceanography, writting in the GMT language
   - __[STAMPS](https://github.com/dbekaert/StaMPS)__ for advanced time-series analysis
   - Various tools for atmospheric corrections: __[TRAIN](https://github.com/dbekaert/TRAIN)__, __[GACOS]()__, __[RAiDER](https://github.com/dbekaert/RAiDER)__ (which I have worked on), and __[PyAPS](http://earthdef.caltech.edu/projects/pyaps/wiki/Main)__.
   - __[MintPy](https://github.com/insarlab/MintPy)__ for InSAR time-series analysis (__[Mintpy tutorial)](https://github.com/insarlab/MintPy-tutorial)__
   - __[PyRate](https://github.com/GeoscienceAustralia/PyRate)__ is a Python tool for InSAR time-series analysis
   - __[GIAnT](http://earthdef.caltech.edu/projects/giant/wiki)__ is yet another InSAR time-series package written in Python
   
__[UNAVCO](https://www.unavco.org/)__ has a number of __[educational resources](https://www.unavco.org/education/education.html)__ available for learning to use the general-purpose InSAR processors, including a __[short courses](https://www.unavco.org/education/professional-development/short-courses/short-courses.html)__, one of which will be __[taught this year](https://www.unavco.org/education/professional-development/short-courses/2020/insar-theory-processing/insar-theory-processing.html)__, and __[past courses](https://www.unavco.org/education/professional-development/short-courses/course-materials/course-materials.html)__ as well. 

<img src="https://raw.githubusercontent.com/jlmaurer/GE6146/master/notebooks/images/UNAVCO_homescreen.png" alt="UNAVCO home page" width="700">

For this lab we will use a __[shared Google Drive folder](https://drive.google.com/drive/folders/1hm3FD0vQXbdvaR2wMXg3GC9AYgYTYBj6?usp=sharing)__ so that you do not need to separately download data. I have already download the data required for this lab to the data/ folder inside the linked shared folder, so you can directly access that. Python programs required for this lab are also in the shared folder so that they are accessible to us while we go through the lab. 

**Each of you have a read/write/edit Google folder inside the shared folder, where you can save intermediate products or results, or even save a copy of this notebook. If you need to save or write data during this lab, use this folder!**

## Single-interferogram analysis versus time-series
One of the most important considerations to make when planning an analysis is whether or not you will need to perform InSAR time-series analysis. We will perform examples of each in this lab to give you a flavor of how they work, but be aware that we will not be able to cover many of the details that are critical for doing a robust full analysis. 

In [2]:
from google.colab import drive
drive.mount('/gdrive')

ModuleNotFoundError: No module named 'google'

In [8]:
# Change to the ARIAtools directory on the shared Google Drive
!cd /gdrive/My\ Drive/GE6371/

In [ ]:
# Import some libraries to be used later
import os
import numpy as np
from osgeo import gdal, ogr

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, FormatStrFormatter, StrMethodFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

!pip install 